# Preamble

This notebook collects analysis and visualization of all data related to nitrate inventories and other mixed-layer properties as compiled in [the previous notebook]().

All figures exported from this notebook are prefixed with `FIGURE_NO3-COMP_`.

In [ ]:
%load_ext autoreload
%autoreload 2
%run imports.py

def load_data():
    # Call signature: df_per_station, df = load_data()
    df_per_station = pd.read_csv('../data/no3-compilation/database-per-stn.csv', parse_dates=['date'])
    df = pd.read_csv('../data/no3-compilation/database.csv', parse_dates=['date'])
    return df_per_station, df

# Define dimensions
dims = dict(
    reg_name=hv.Dimension('reg_name', label='Region'),
    ntr0=hv.Dimension('ntr0', label='Surface NO₃', unit='µM'),
    doy=hv.Dimension('doy', label='Day of the year', range=(0,360)),
    lon=hv.Dimension('lon', label='Longitude', unit='°E'),
    lat=hv.Dimension('lat', label='Latitude', unit='°N'),
    delta_sigth=hv.Dimension('delta_sigth', label='Δσ-θ', unit='kg m⁻³'),
    strat=hv.Dimension('strat', label='N²', unit='s⁻²'),
    logstrat=hv.Dimension('logstrat', label='log10 N²', unit='s⁻²'),
    no3_sfc_winter = hv.Dimension('no3_sfc_winter', label='Pre-bloom sfc. NO₃ ', unit='µM', range=(-.5,13))
)

# Maps of surface nitrate conc.

In [ ]:
df, dfp = load_data()

def finalize_mpl(fig):
    fig.set_size_inches(8,8)
    ax = fig.axes[0]
    ax.set_title(dims['ntr0'].pprint_label)
    ax = fig.axes[1]
    ax.yaxis.set_label_text('')

options = [
    opts.HexTiles(
        cmap=cmocean.cm.matter, projection=ccrs.NorthPolarStereo(), gridsize=20, colorbar=True,
        aspect=1, aggregator=np.mean, tools=['hover'],
        # clipping_colors={'nan': 'grey'},
    ),
    opts.Feature(scale='50m', projection=ccrs.NorthPolarStereo()),
    opts.Overlay(bgcolor='grey')
]

l = lambda season: (
    gv.HexTiles(df.loc[df.season==season].dropna(subset=['ntr0']), ['lon', 'lat'], 'ntr0')
    * land
).opts(*(options+translate_options(options, bokeh2mpl))).redim(**dims)

for season in ['winter', 'summer']:
    fname = '../nb_fig/FIGURE_NO3-COMP_map_no3_'+season
    obj = l(season)
    hv.save(obj, fname, fmt='html')
    fig = mplrender_map(obj, fname=fname, hooks=[finalize_mpl])
    # hv.output(obj)
    
fig

# Seasonality

In [ ]:
df, dfp = load_data()

## Seasonality by regions

In [ ]:
loc = gv.Points(df, ['lon', 'lat'], [])
ntr = hv.Scatter(df, 'doy', ['ntr0'])

DataLink(loc,ntr)

l = loc*land*gv.feature.rivers + ntr 
l = l.redim(**dims).cols(2)

options = (
    opts.Scatter(
        tools=['lasso_select', 'box_select'], active_tools=['box_select'], 
        color='lightblue', size=5, selection_color='orange',
        frame_height=250, frame_width=500
    ),
    opts.Points(
        tools=['lasso_select', 'box_select'], active_tools=['box_select'],
        projection=ccrs.NorthPolarStereo(),
        color='lightblue', size=3, selection_color='orange',
        frame_height=250, aspect=1, 
        # align='center'
    ),
    opts.Feature(
        fill_color='wheat', line_color='black',
        scale='50m'
    ),
    opts.Feature('Rivers', line_color='blue', scale='110m'),
)
l = l.opts(*options).redim.range(ntr0=(0, 15))

hv.save(l, '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle.html')
l

## Make static matplotlib plot for paper 

In [ ]:
sc = df.loc[~df.reg_name.isnull()].hvplot.scatter('doy', 'ntr0', groupby='reg_name').layout().cols(2)
err = bin_average(sc, bins=np.linspace(0,366,13))
l = sc * err * err.map(hv.Curve, hv.Dataset) * err.map(hv.Scatter, hv.Dataset)

In [ ]:
options = [
    opts.Scatter(
        frame_width=300, frame_height=150, 
        size=3, alpha=.4, axiswise=True,
        color='g', show_legend=False,
    ),
    opts.ErrorBars(color='k', alpha=.3,),
    opts.Curve(color='k'),
    opts.NdLayout(title='A')
]

l = l.redim(**dims).opts(*options)

fname = '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle_wo_map'
hv.save(l.opts(toolbar=None, clone=True), fname, fmt='png')

l

In [ ]:
subprocess.run([
    'convert',
    '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle_wo_map.png',
    '../nb_fig/FIGURE_NO3-COMP_regions.png',
    '-append',
    '../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle.png'
])

!rm ../nb_fig/FIGURE_NO3-COMP_chart_seasonal_cycle_wo_map.png

!rm ../nb_fig/FIGURE_NO3-COMP_regions.png